In [1]:
%load_ext autoreload
%autoreload 2

import helpers as s

import OAI

In [2]:
import os, openai, datetime, hashlib, re
import time
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import pandas as pd


from dotenv import load_dotenv
load_dotenv()

True

In [3]:
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))



OPENAI_API_KEY is ready


/home/kelu/projets/substack/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
signals = pd.read_parquet("data/signals.parquet.gzip")
metatags = pd.read_parquet("data/metatags.parquet.gzip")

In [5]:
df = pd.read_parquet('data/articles.parquet.gzip')
df.columns=  ["src","content","LEN"]
df = df[(df.LEN > 1500) & (df.LEN < 30000)].reset_index(drop=True)
titles = pd.read_parquet("data/titles.parquet.gzip")
df = df.merge(titles,on="src",how="left")
mt = pd.read_parquet("data/metatags.parquet.gzip")
df = df.merge(mt,on="src",how="left")
df.to_parquet("data/consolidated.parquet.gzip",compression="gzip")

In [6]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [7]:
articles = intersection(metatags.src.unique(), signals.src.unique() )
signals = signals[signals.src.isin(articles)]
metatags = metatags[metatags.src.isin(articles)]
len(articles)

1115

It takes 4 mins for 1000 articles. Approx.

In [8]:
base_path = "./DB/"

if not os.path.isfile(base_path+"chroma.sqlite3"):
    print("Start a new DB")
    vectordb = Chroma.from_documents(
        documents=[chunked_documents[0]],
        embedding=embeddings,
        persist_directory=base_path
    )
    vectordb.persist()
else:
    print("Continue on the DB")
    vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
    print(len(vectordb.get()["ids"]),"elements already stored.")
    LSDOCS = vectordb.get()["documents"]

Continue on the DB
1410 elements already stored.


In [9]:
def getClosest(txt):
    B = vectordb.similarity_search(txt,6)
    L = []
    X = []
    for b in list(B):
        #print(b.metadata["src"],b.metadata["title"])
        L.append([b.metadata["src"],b.metadata["title"]])
        X.append(b.metadata["src"])
    return X

In [16]:
from tqdm import tqdm
from tqdm.gui import tqdm as tqdm_gui
tqdm.pandas(ncols=50)

In [17]:
metatags["closest"] = ""
metatags["closest"] = metatags["summary"].apply(lambda x: getClosest(x))
metatags

,src,keywords,themes,summary,origin,url,closest
0,c94f72ff677c7517a836417c1f1df620,"[AI, work, consultants, tasks, performance, Ja...","[Impact of AI on work, The Jagged Frontier of ...","In this post, the author discusses the impact ...",20230927,b'https://www.oneusefulthing.org/p/centaurs-an...,"[c94f72ff677c7517a836417c1f1df620, c63bd059cb5..."
2,656239ad8ab36ce06ac4ee6258b5381d,"[Atlas for The Blind, 1837, blind, embossed, p...","[education, disabilities, geography]","The Atlas for the Blind, published in 1837, wa...",20230616,b'https://www.davidrumsey.com/blog/2012/5/21/a...,"[656239ad8ab36ce06ac4ee6258b5381d, a4764a55658..."
3,94960bd3c0ad4c594fd56651f4087128,"[drinking habits, generation sensible, alcohol...","[sober movement, reasons for going sober, impa...",The trend of young people turning away from al...,20220922,b'https://www.huckmag.com/perspectives/the-rea...,"[94960bd3c0ad4c594fd56651f4087128, d64c4666be4..."
4,d2a5e5d3dad76d8880d6d322b26a8ab9,"[startup, plastic-eating enzymes, waste, Epoch...","[Sustainability, Plastic waste, Synthetic biol...","Epoch Biodesign, a startup, has raised $11m to...",20230108,b'https://sifted.eu/articles/epoch-biodesign-e...,"[d2a5e5d3dad76d8880d6d322b26a8ab9, 652e3b2a7f8..."
5,5c135d085a6d575c9d112244b2a873fb,"[proprietary software, computing freedom, Serv...","[proprietary software, computing freedom, SaaSS]","In this article, Richard Stallman discusses th...",20220810,b'https://www.gnu.org/philosophy/who-does-that...,"[5c135d085a6d575c9d112244b2a873fb, 62f0c80f009..."
...,...,...,...,...,...,...,...
1398,c346127b59f55385f443e5e27e93bc78,"[private equity acquisition, quality of care, ...","[private equity acquisition of hospitals, hosp...",private equity acquisition of hospitals is ass...,20240204,b'https://jamanetwork.com/journals/jama/articl...,"[c346127b59f55385f443e5e27e93bc78, 86944a8ff63..."
1399,012558fa97aeed65d7ca86de4a6c1cbd,"[Department of Defense, Chief Digital and Arti...","[Bias detection in AI systems, Crowdsourced au...",The Department of Defense (DoD) Chief Digital ...,20240210,b'https://www.defense.gov/News/Releases/Releas...,"[012558fa97aeed65d7ca86de4a6c1cbd, fab122d29ae..."
1403,a6bfcabefea560d7dfefd5aab87f047c,"[physicist, nanoscience lab, cooling paint, vi...","[nanoscience, structural color, environmentall...",Debashis Chanda and his team at the University...,20230408,b'https://www.wired.com/story/lightest-paint-i...,"[a6bfcabefea560d7dfefd5aab87f047c, 8e1b3608f46..."
1404,5d04744a3bee2ab6564f5e2ff60136d8,"[knowledge graphs, unstructured text, LLMs, da...","[Knowledge extraction, Unstructured data proce...",This blog post discusses the use of Large Lang...,20231017,b'https://medium.com/neo4j/construct-knowledge...,"[5d04744a3bee2ab6564f5e2ff60136d8, 69aa55d9702..."


In [18]:
metatags.to_parquet("data/pages_metatags_with_closest.parquet.gzip", compression="gzip")

# Closest done

In [19]:
metatags = pd.read_parquet("data/pages_metatags_with_closest.parquet.gzip")
metatags["kwsl"] = metatags["keywords"].apply(lambda lst: [x.lower().replace('"','') for x in lst])
metatags = metatags.merge(df[["src","title"]],on="src",how="left")
metatags

,src,keywords,themes,summary,origin,url,closest,kwsl,title
0,c94f72ff677c7517a836417c1f1df620,"[AI, work, consultants, tasks, performance, Ja...","[Impact of AI on work, The Jagged Frontier of ...","In this post, the author discusses the impact ...",20230927,b'https://www.oneusefulthing.org/p/centaurs-an...,"[c94f72ff677c7517a836417c1f1df620, c63bd059cb5...","[ai, work, consultants, tasks, performance, ja...",Centaurs and Cyborgs on the Jagged Frontier
1,656239ad8ab36ce06ac4ee6258b5381d,"[Atlas for The Blind, 1837, blind, embossed, p...","[education, disabilities, geography]","The Atlas for the Blind, published in 1837, wa...",20230616,b'https://www.davidrumsey.com/blog/2012/5/21/a...,"[656239ad8ab36ce06ac4ee6258b5381d, a4764a55658...","[atlas for the blind, 1837, blind, embossed, p...",Atlas for The Blind 1837
2,94960bd3c0ad4c594fd56651f4087128,"[drinking habits, generation sensible, alcohol...","[sober movement, reasons for going sober, impa...",The trend of young people turning away from al...,20220922,b'https://www.huckmag.com/perspectives/the-rea...,"[94960bd3c0ad4c594fd56651f4087128, d64c4666be4...","[drinking habits, generation sensible, alcohol...",The Rise of Sober Generation
3,d2a5e5d3dad76d8880d6d322b26a8ab9,"[startup, plastic-eating enzymes, waste, Epoch...","[Sustainability, Plastic waste, Synthetic biol...","Epoch Biodesign, a startup, has raised $11m to...",20230108,b'https://sifted.eu/articles/epoch-biodesign-e...,"[d2a5e5d3dad76d8880d6d322b26a8ab9, 652e3b2a7f8...","[startup, plastic-eating enzymes, waste, epoch...",Startup Raises $11m to Develop Plastic-Eating ...
4,5c135d085a6d575c9d112244b2a873fb,"[proprietary software, computing freedom, Serv...","[proprietary software, computing freedom, SaaSS]","In this article, Richard Stallman discusses th...",20220810,b'https://www.gnu.org/philosophy/who-does-that...,"[5c135d085a6d575c9d112244b2a873fb, 62f0c80f009...","[proprietary software, computing freedom, serv...",The Dangers of Service as a Software Substitut...
...,...,...,...,...,...,...,...,...,...
1110,c346127b59f55385f443e5e27e93bc78,"[private equity acquisition, quality of care, ...","[private equity acquisition of hospitals, hosp...",private equity acquisition of hospitals is ass...,20240204,b'https://jamanetwork.com/journals/jama/articl...,"[c346127b59f55385f443e5e27e93bc78, 86944a8ff63...","[private equity acquisition, quality of care, ...",Quality of Care and Patient Outcomes After Pri...
1111,012558fa97aeed65d7ca86de4a6c1cbd,"[Department of Defense, Chief Digital and Arti...","[Bias detection in AI systems, Crowdsourced au...",The Department of Defense (DoD) Chief Digital ...,20240210,b'https://www.defense.gov/News/Releases/Releas...,"[012558fa97aeed65d7ca86de4a6c1cbd, fab122d29ae...","[department of defense, chief digital and arti...",DoD Launches AI Bias Bounty Exercises
1112,a6bfcabefea560d7dfefd5aab87f047c,"[physicist, nanoscience lab, cooling paint, vi...","[nanoscience, structural color, environmentall...",Debashis Chanda and his team at the University...,20230408,b'https://www.wired.com/story/lightest-paint-i...,"[a6bfcabefea560d7dfefd5aab87f047c, 8e1b3608f46...","[physicist, nanoscience lab, cooling paint, vi...",Revolutionary Light and Cool Structural Paint
1113,5d04744a3bee2ab6564f5e2ff60136d8,"[knowledge graphs, unstructured text, LLMs, da...","[Knowledge extraction, Unstructured data proce...",This blog post discusses the use of Large Lang...,20231017,b'https://medium.com/neo4j/construct-knowledge...,"[5d04744a3bee2ab6564f5e2ff60136d8, 69aa55d9702...","[knowledge graphs, unstructured text, llms, da...",Extracting Knowledge from Unstructured Text


In [20]:
metatags[metatags.src == "feeb207dfea30efb1d5bf73503fd22a4"]["closest"].iloc[0]

array(['feeb207dfea30efb1d5bf73503fd22a4',
       'df59e2cf3380ffd9aeac9a3e01073300',
       'ed237776f4979a2104f62c4985fbeba8',
       'c42a95f16678ed3834840d48f8e775a3',
       '767b74c90576473294b2c47568c0e355',
       '5cf4407dc6fa3889e047c658e27c4ccf'], dtype=object)

In [21]:
for ix, row in metatags.iterrows():
    try:
        ID = row["src"]
        TXT = "# __"+row["title"] +"__, from (["+row.origin+"](https://kghosh.substack.com/p/"+row.origin+").)\n\n"
        TXT += "__[External link]("+str(row.url)[2:-1]+")__\n\n"
        TXT += "\n\n## Summary\n\n"
        TXT += row["summary"] + "\n\n## Keywords\n\n"
        TXT += "* "+"\n* ".join(row["keywords"])
        TXT += "\n\n## Themes\n\n"
        TXT += "* "+"\n* ".join(row["themes"])
        TXT += "\n\n## Signals\n\n"    
        SIG = signals[signals.src == ID]
        TXT += SIG[list(SIG.columns)[:-1]].to_markdown(index=False)
        if 1:
            TXT += "\n\n## Closest\n\n"
            TXT += "* "+"\n* ".join([ "["+df.loc[df["src"] == x,"title"].iloc[0]+"]("+x+")" for x in row["closest"][1:]])
        with open("docs/"+ID+".md", "w") as f:
            f.write(TXT) 
    except:
        print("Error with",row["src"])


# Now creating index

In [22]:
import random

In [23]:
allkw = []
for kw in metatags.themes:# keywords: #as an alternative
    allkw = allkw+list(kw)
KW = list(set(allkw))
KW = list(set([x.capitalize().replace('"','') for x in KW]))
KW = [x for x in KW if not x.isnumeric()]
KW.sort()
print(len(KW))
random.shuffle(KW)
KW[:10]

2763


['Brexit impact',
 'Greenlandic support for sand extraction',
 'Importance of community and social interaction in the workplace',
 'Rise of pickleball',
 'Usb protocols',
 'Education technology',
 'Benefits and challenges of generative ai',
 'Student loans',
 'Chatgpt impact on book industry',
 'Wearable medical devices']

In [24]:
h = OAI.Helper("local_seeds_classif") 

In [25]:
with open("data/categories_small.csv","r") as f:
    ALL = f.read().split("\n")
ALL.sort()
ALL

['3D printing',
 'AI Applications',
 'AI Impacts',
 'AI Systems and Technology',
 'AI and Pop Culture ',
 'AI regulations and EU policies',
 'API development',
 'Access to culture',
 'Access to services',
 'Accounting',
 'Adoption and Use Cases of AI',
 'Adoption of generative AI',
 'Aerodynamics and engineering principles',
 'Affordability and accessibility of broadband services',
 'Aging and societal changes',
 'Agriculture and farming',
 'Airline Operations and Policies',
 'Algorithmic Model Optimization',
 'Animal behavior and cognition',
 'Animal communication',
 'Archaeological Discoveries and Research',
 'Architecture and social thought',
 'Art and Creativity',
 'Astronomy',
 'Attention Economy',
 'Automation',
 'Autonomous Systems and Agents',
 'Awards and recognition',
 'Branding and advertising',
 'Business and Economy',
 'Capitalism ',
 'Car subscriptions and mobility solutions',
 'Cartography',
 'Challenges ',
 'Chemistry and Composition',
 'Civic engagement and activism',


In [26]:
metatags

,src,keywords,themes,summary,origin,url,closest,kwsl,title
0,c94f72ff677c7517a836417c1f1df620,"[AI, work, consultants, tasks, performance, Ja...","[Impact of AI on work, The Jagged Frontier of ...","In this post, the author discusses the impact ...",20230927,b'https://www.oneusefulthing.org/p/centaurs-an...,"[c94f72ff677c7517a836417c1f1df620, c63bd059cb5...","[ai, work, consultants, tasks, performance, ja...",Centaurs and Cyborgs on the Jagged Frontier
1,656239ad8ab36ce06ac4ee6258b5381d,"[Atlas for The Blind, 1837, blind, embossed, p...","[education, disabilities, geography]","The Atlas for the Blind, published in 1837, wa...",20230616,b'https://www.davidrumsey.com/blog/2012/5/21/a...,"[656239ad8ab36ce06ac4ee6258b5381d, a4764a55658...","[atlas for the blind, 1837, blind, embossed, p...",Atlas for The Blind 1837
2,94960bd3c0ad4c594fd56651f4087128,"[drinking habits, generation sensible, alcohol...","[sober movement, reasons for going sober, impa...",The trend of young people turning away from al...,20220922,b'https://www.huckmag.com/perspectives/the-rea...,"[94960bd3c0ad4c594fd56651f4087128, d64c4666be4...","[drinking habits, generation sensible, alcohol...",The Rise of Sober Generation
3,d2a5e5d3dad76d8880d6d322b26a8ab9,"[startup, plastic-eating enzymes, waste, Epoch...","[Sustainability, Plastic waste, Synthetic biol...","Epoch Biodesign, a startup, has raised $11m to...",20230108,b'https://sifted.eu/articles/epoch-biodesign-e...,"[d2a5e5d3dad76d8880d6d322b26a8ab9, 652e3b2a7f8...","[startup, plastic-eating enzymes, waste, epoch...",Startup Raises $11m to Develop Plastic-Eating ...
4,5c135d085a6d575c9d112244b2a873fb,"[proprietary software, computing freedom, Serv...","[proprietary software, computing freedom, SaaSS]","In this article, Richard Stallman discusses th...",20220810,b'https://www.gnu.org/philosophy/who-does-that...,"[5c135d085a6d575c9d112244b2a873fb, 62f0c80f009...","[proprietary software, computing freedom, serv...",The Dangers of Service as a Software Substitut...
...,...,...,...,...,...,...,...,...,...
1110,c346127b59f55385f443e5e27e93bc78,"[private equity acquisition, quality of care, ...","[private equity acquisition of hospitals, hosp...",private equity acquisition of hospitals is ass...,20240204,b'https://jamanetwork.com/journals/jama/articl...,"[c346127b59f55385f443e5e27e93bc78, 86944a8ff63...","[private equity acquisition, quality of care, ...",Quality of Care and Patient Outcomes After Pri...
1111,012558fa97aeed65d7ca86de4a6c1cbd,"[Department of Defense, Chief Digital and Arti...","[Bias detection in AI systems, Crowdsourced au...",The Department of Defense (DoD) Chief Digital ...,20240210,b'https://www.defense.gov/News/Releases/Releas...,"[012558fa97aeed65d7ca86de4a6c1cbd, fab122d29ae...","[department of defense, chief digital and arti...",DoD Launches AI Bias Bounty Exercises
1112,a6bfcabefea560d7dfefd5aab87f047c,"[physicist, nanoscience lab, cooling paint, vi...","[nanoscience, structural color, environmentall...",Debashis Chanda and his team at the University...,20230408,b'https://www.wired.com/story/lightest-paint-i...,"[a6bfcabefea560d7dfefd5aab87f047c, 8e1b3608f46...","[physicist, nanoscience lab, cooling paint, vi...",Revolutionary Light and Cool Structural Paint
1113,5d04744a3bee2ab6564f5e2ff60136d8,"[knowledge graphs, unstructured text, LLMs, da...","[Knowledge extraction, Unstructured data proce...",This blog post discusses the use of Large Lang...,20231017,b'https://medium.com/neo4j/construct-knowledge...,"[5d04744a3bee2ab6564f5e2ff60136d8, 69aa55d9702...","[knowledge graphs, unstructured text, llms, da...",Extracting Knowledge from Unstructured Text


In [27]:
import json
with open("data/categories_ax.json","r") as f:
    ax = json.loads(f.read())
with open("data/categories_bx.json","r") as f:
    bx = json.loads(f.read())

In [28]:

for ix, row in metatags.iterrows():
    if ix < 100000: # useless
        ax[row["src"]] = {}
        if row["src"] not in ax.keys():
            PROMPT = "You are given a text below. You need to pick the five most relevant themes of this text from the list of keywords below:\n\n* "+"\n*".join(ALL)+"\n\n\nAnswer only with 3 items of this list, separated by commas"
            ANS =  h.ask(PROMPT,row["summary"],src="tag",v="gpt-3.5-turbo-0125")
            LST = [x.strip() for x in ANS.split(",") if x.strip() in ALL]
            ax[row["src"]] = LST
            for x in LST:
                if x not in bx.keys():
                    bx[x] = []
                bx[x].append(row["src"])

In [29]:
import json
with open("data/categories_ax.json","w") as f:
    f.write(json.dumps(ax))
with open("data/categories_bx.json","w") as f:
    f.write(json.dumps(bx))

In [30]:
import string
LETTERS = string.ascii_uppercase
LETTERS

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [31]:
bx['Epidemic Outbreaks and Response'][0]

'b1b5f0564e486b3e1f123da956f6b1a6'

In [32]:
INDEX  = "# Overview \n\n"
INDEX += "This page tries to capture the information obtained in my [tech watch](https://substack.kghosh.me/). The different links and pages are captured below, trying to catalogue different weak signals." 


for letter in LETTERS:
    kws = [x for x in list(bx.keys()) if x[0] == letter]

    if len(kws):
        INDEX += "\n\n## Letter '"+letter.upper()+"' \n\n"
        for kw in kws:
            INDEX += "\n### Category: __"+kw+"__: \n\n"
            lst = bx[kw] 
            for x in lst:
                TITRE = df.loc[df.src == x,"title"].iloc[0]
                INDEX += "* ["+TITRE+"]("+x+")\n"

INDEX += "\n\n## Others\n\n"
for x in ax:
    if len(ax[x]) == 0:
        TITRE = df.loc[df.src == x,"title"].iloc[0]
        INDEX += "* ["+TITRE+"]("+x+")\n"

In [33]:
with open("docs/index.md", "w") as f:
    f.write(INDEX) 